<a href="https://colab.research.google.com/github/kovus380/PS/blob/master/category_classifier_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip install konlpy

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,431 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http:

In [ ]:
import pickle
with open('/content/original_product_data.pkl', 'rb') as fr:
    data = pickle.load(fr)
print(data)

In [132]:
category_label = [0, 0, 0, 1, 1, 1, 3, 1, 1, 1, 5, 3, 3, 3, 3, 6, 4, 4, 4, 4, 5, 6, 2, 2, 2, 6, 2, 2 ]

In [ ]:
print(len(category_label))

In [ ]:
data

In [135]:
import numpy as np
for idx, value in enumerate(category_label):
    data['label'] = np.select([data.label == idx], [value], data.label) 

In [ ]:
data[1555:5000]

In [ ]:
data

In [138]:
data = data.loc[:, ["name", "label"]]

In [ ]:
data

In [140]:
data['name'] = data['name'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
data['name'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경

In [ ]:
data

In [ ]:
print(len(data))

In [ ]:
data.isnull().sum()

In [ ]:
data.dropna(how='any')

In [145]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import pandas as pd 

# split하면서 shuffle 적용
product_train, product_test, y_train, y_test = train_test_split(data['name'], data['label'], test_size=0.2, random_state = 42)

# 원핫벡터로 만들어줍시다! (num_classes로 카테고리 수 명시 가능)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

product_train = pd.DataFrame(product_train)
product_test = pd.DataFrame(product_test)

In [ ]:
product_train

In [ ]:
print(len(y_train))
print(len(y_test))

In [148]:
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게', '인기', '짱', '강력', '추천', '선물', '특가', '할인', '세일', '생일', '기념일']

In [149]:
product_train = product_train.astype(
    {'name': str}
)

product_test = product_test.astype(
    {'name': str}
)

In [150]:
from konlpy.tag import Okt
import matplotlib.pyplot as plt

In [151]:
okt = Okt()

In [152]:
product_train['tokenized'] = product_train['name'].apply(okt.morphs)
product_train['tokenized'] = product_train['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])

product_test['tokenized'] = product_test['name'].apply(okt.morphs)
product_test['tokenized'] = product_test['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])

In [ ]:
product_train

In [ ]:
product_test

In [156]:
product_train = product_train['tokenized'].values
product_test = product_test['tokenized'].values

In [ ]:
print(product_train)

In [158]:
import csv 
# vocab_size = 42019
# X_train = []
path = '/content/category_train.csv'
with open(path, 'w') as f:
    writer = csv.writer(f)
    writer.writerows(product_train)

In [ ]:
print(len(product_train))
print(len(product_test))

In [160]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [161]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(product_train)

In [ ]:
print(product_train)

In [ ]:
print(tokenizer.word_index)

In [164]:
total_count = len(tokenizer.word_index)

In [ ]:
print(total_count)

In [ ]:
print('리뷰의 최대 길이 :',max(len(l) for l in product_train))
print('리뷰의 평균 길이 :',sum(map(len, product_train))/len(product_train))
plt.hist([len(s) for s in product_train], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [167]:
def below_threshold_len(max_len, nested_list):
  cnt = 0
  for s in nested_list:
    if(len(s) <= max_len):
        cnt = cnt + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))*100))

In [ ]:
max_len = 15
below_threshold_len(max_len, product_train)

In [ ]:
print(len(product_train))

In [ ]:
print(total_count)

In [174]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(total_count)
tokenizer.fit_on_texts(product_train)

product_train = tokenizer.texts_to_sequences(product_train)
product_test = tokenizer.texts_to_sequences(product_test)

In [ ]:
print(tokenizer.word_index)

In [ ]:
product_train[:5]

In [177]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_len = 15
product_train = pad_sequences(product_train, maxlen=max_len)
product_test = pad_sequences(product_test, maxlen=max_len)

In [178]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [ ]:
print(len(product_test))
print(len(y_test))

In [ ]:
print(product_test[1])
print(y_test[1])

In [ ]:
print(len(product_train[0]))
print(len(product_test[0]))

print(len(y_test[0]))
print(len(y_train[0]))


In [ ]:
y_train.shape

In [ ]:
y_test.shape

In [184]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [187]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('category_best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [ ]:
model = Sequential()
model.add(Embedding(25000, 128))
model.add(LSTM(64))
model.add(Dense(7, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(product_train, y_train, callbacks=[es, mc], validation_data=(product_test, y_test), batch_size=32, epochs=60)

In [189]:
category_dict = {0: '먹거리', 1: '패션', 2: '잡화', 3: '쥬얼리', 4: '인테리어 소품', 5: '생활', 6: '기타'}

In [190]:
def product_predict(product):
    product = okt.morphs(product)
    product = [word for word in product if not word in stopwords]
    print(product)
    encoded = tokenizer.texts_to_sequences([product])
    print(encoded)
    pad_new = pad_sequences(encoded, maxlen=max_len)
    print(pad_new)
    score = (model.predict(pad_new)).astype(float)
    print(score)
    print(category_dict[score.argmax()] if score.max() > 0.5 else category_dict[28])

In [ ]:
product_predict('맛있는 마카롱')

In [ ]:
product_predict('녹차비누유기농 그린티솝')

In [ ]:
product_predict('달 무드등')